## Use online learning algorithms as batch learning algorithms


In [1]:
from add_noise import add_noise
from gen import gen
import numpy as np
import math
import matplotlib.pyplot as plt

In [2]:
def perceptron(x, y):
    eta = 1
    m, n = x.shape
    w = np.zeros(n)
    theta = 0
    for i in range(20):
        for j in range(m):
            if (np.dot(w, x[j, :]) + theta) * y[j] <= 0:
                w += eta * y[j] * x[j, :]
                theta += eta * y[j]
    return w, theta

def perceptron_margin(x, y, eta):
    m, n = x.shape
    w = np.zeros(n)
    theta = 0
    for i in range(20):
        for j in range(m):
            if (np.dot(w, x[j, :]) + theta) * y[j] <= 1:
                w += eta * y[j] * x[j, :]
                theta += eta * y[j]
    return w, theta

def winnow(x, y, alpha):
    m, n = x.shape
    w = np.ones(n)
    theta = -n
    for i in range(20):
        for j in range(m):
            if (np.dot(w, x[j, :]) + theta) * y[j] <= 0:
                for k in range(n):
                    w[k] = w[k] * alpha**(y[j]*x[j, k])
    return w, theta

def winnow_margin(x, y, alpha, gamma):
    m, n = x.shape
    w = np.ones(n)
    theta = -n
    for i in range(20):
        for j in range(m):
            if (np.dot(w, x[j, :]) + theta) * y[j] <= gamma:
                for k in range(n):
                    w[k] = w[k] * alpha**(y[j]*x[j, k])
    return w, theta

def adagrad(x, y, eta):
    m, n = x.shape
    w = np.append(np.ones(n), [-n])
    G = np.zeros(n+1)

    for i in range(20):
        for j in range(m):
            if (np.dot(w[:n], x[j, :]) + w[n]) * y[j] <= 1:
                # that gradient is -yx, and for $\theta$, it is always -y.
                g = np.append(-y[j] * x[j, :], -y[j])
                G += np.square(g)
                for k in range(n+1):
                    if G[k] != 0:
                        w[k] = w[k] - eta * g[k] / np.sqrt(G[k])
    return w[:n], w[n]

def accuracy(w, theta, x, y):
    correct = 0
    m, _ = x.shape
    for i in range(m):
        if (np.dot(w, x[i, :]) + theta) * y[i] >=0:
            correct += 1
    return 1.0*correct/m



In [4]:
perceptron_params = [1.5,0.25,0.03,0.005,0.001]
winnow_params = [1.1,1.01,1.005,1.0005,1.0001]
winnow_margin_params1 = [1.1,1.01,1.005,1.0005,1.0001]
winnow_margin_params2 = [2.0,0.3,0.04,0.006,0.001]
adagrad_params = [1.5,0.25,0.03,0.005,0.001]

In [5]:
train_y, train_x = [], []
test_y, test_x = [], []

yTrain,xTrain = gen(10,100,1000,50000,True)
train_y.append(yTrain)
train_x.append(xTrain)

yTest,xTest = gen(10,100,1000,10000,False)
test_y.append(yTest)
test_x.append(xTest)


yTrain,xTrain = gen(10,500,1000,50000,True)
train_y.append(yTrain)
train_x.append(xTrain)

yTest,xTest = gen(10,500,1000,10000,False)
test_y.append(yTest)
test_x.append(xTest)

yTrain,xTrain = gen(10,1000,1000,50000,True)
train_y.append(yTrain)
train_x.append(xTrain)

yTest,xTest = gen(10,1000,1000,10000,False)
test_y.append(yTest)
test_x.append(xTest)

In [6]:
m = [100, 500, 1000]
for i in range(3):
    w,theta = perceptron(train_x[i][:5000,:], train_y[i][:5000])
    acc = accuracy(w,theta,train_x[i][5000:10000,:],train_y[i][5000:10000])
    print("Perceptron : m = {}, accuracy = {}".format(m[i], acc))

Perceptron : m = 100, accuracy = 0.7446
Perceptron : m = 500, accuracy = 0.593
Perceptron : m = 1000, accuracy = 0.7352


In [7]:
for i in range(3):
    for eta in perceptron_params:
        w,theta = perceptron_margin(train_x[i][:5000,:], train_y[i][:5000], eta)
        acc = accuracy(w,theta,train_x[i][5000:10000,:],train_y[i][5000:10000])
        print("Perceptron margin : m = {}, eta = {} accuracy = {}".format(m[i], eta, acc))

Perceptron margin : m = 100, eta = 1.5 accuracy = 0.7446
Perceptron margin : m = 100, eta = 0.25 accuracy = 0.7374
Perceptron margin : m = 100, eta = 0.03 accuracy = 0.736
Perceptron margin : m = 100, eta = 0.005 accuracy = 0.7104
Perceptron margin : m = 100, eta = 0.001 accuracy = 0.6816
Perceptron margin : m = 500, eta = 1.5 accuracy = 0.593
Perceptron margin : m = 500, eta = 0.25 accuracy = 0.5532
Perceptron margin : m = 500, eta = 0.03 accuracy = 0.571
Perceptron margin : m = 500, eta = 0.005 accuracy = 0.6096
Perceptron margin : m = 500, eta = 0.001 accuracy = 0.5492
Perceptron margin : m = 1000, eta = 1.5 accuracy = 0.7352
Perceptron margin : m = 1000, eta = 0.25 accuracy = 0.7476
Perceptron margin : m = 1000, eta = 0.03 accuracy = 0.7328
Perceptron margin : m = 1000, eta = 0.005 accuracy = 0.63
Perceptron margin : m = 1000, eta = 0.001 accuracy = 0.5752


In [8]:
for i in range(3):
    for alpha in winnow_params:
        w,theta = winnow(train_x[i][:5000,:], train_y[i][:5000], alpha)
        acc = accuracy(w,theta,train_x[i][5000:10000,:],train_y[i][5000:10000])
        print("Winnow : m = {}, alpha = {} accuracy = {}".format(m[i], alpha, acc))

Winnow : m = 100, alpha = 1.1 accuracy = 0.8702
Winnow : m = 100, alpha = 1.01 accuracy = 0.748
Winnow : m = 100, alpha = 1.005 accuracy = 0.5938
Winnow : m = 100, alpha = 1.0005 accuracy = 0.5118
Winnow : m = 100, alpha = 1.0001 accuracy = 0.5076
Winnow : m = 500, alpha = 1.1 accuracy = 0.786
Winnow : m = 500, alpha = 1.01 accuracy = 0.5286
Winnow : m = 500, alpha = 1.005 accuracy = 0.5112
Winnow : m = 500, alpha = 1.0005 accuracy = 0.5058
Winnow : m = 500, alpha = 1.0001 accuracy = 0.5086
Winnow : m = 1000, alpha = 1.1 accuracy = 0.7386
Winnow : m = 1000, alpha = 1.01 accuracy = 0.6462
Winnow : m = 1000, alpha = 1.005 accuracy = 0.4992
Winnow : m = 1000, alpha = 1.0005 accuracy = 0.4992
Winnow : m = 1000, alpha = 1.0001 accuracy = 0.4992


In [9]:
for i in range(3):
    for alpha in winnow_margin_params1:
        for gamma in winnow_margin_params2:
            w,theta = winnow_margin(train_x[i][:5000,:], train_y[i][:5000], alpha, gamma)
            acc = accuracy(w,theta,train_x[i][5000:10000,:],train_y[i][5000:10000])
            print("Winnow : m = {}, alpha = {}, gamma = {} accuracy = {}".format(m[i], alpha, gamma, acc))

Winnow : m = 100, alpha = 1.1, gamma = 2.0 accuracy = 0.8676
Winnow : m = 100, alpha = 1.1, gamma = 0.3 accuracy = 0.8786
Winnow : m = 100, alpha = 1.1, gamma = 0.04 accuracy = 0.8642
Winnow : m = 100, alpha = 1.1, gamma = 0.006 accuracy = 0.8768
Winnow : m = 100, alpha = 1.1, gamma = 0.001 accuracy = 0.8702
Winnow : m = 100, alpha = 1.01, gamma = 2.0 accuracy = 0.7654
Winnow : m = 100, alpha = 1.01, gamma = 0.3 accuracy = 0.7492
Winnow : m = 100, alpha = 1.01, gamma = 0.04 accuracy = 0.7518
Winnow : m = 100, alpha = 1.01, gamma = 0.006 accuracy = 0.7522
Winnow : m = 100, alpha = 1.01, gamma = 0.001 accuracy = 0.751
Winnow : m = 100, alpha = 1.005, gamma = 2.0 accuracy = 0.6002
Winnow : m = 100, alpha = 1.005, gamma = 0.3 accuracy = 0.5942
Winnow : m = 100, alpha = 1.005, gamma = 0.04 accuracy = 0.5946
Winnow : m = 100, alpha = 1.005, gamma = 0.006 accuracy = 0.5948
Winnow : m = 100, alpha = 1.005, gamma = 0.001 accuracy = 0.5964
Winnow : m = 100, alpha = 1.0005, gamma = 2.0 accuracy =

In [10]:
for i in range(3):
    for eta in adagrad_params:
        w,theta = adagrad(train_x[i][:5000,:], train_y[i][:5000], eta)
        acc = accuracy(w,theta,train_x[i][5000:10000,:],train_y[i][5000:10000])
        print("adagrad : m = {}, eta = {}, accuracy = {}".format(m[i], eta, acc))

adagrad : m = 100, eta = 1.5, accuracy = 0.6764
adagrad : m = 100, eta = 0.25, accuracy = 0.556
adagrad : m = 100, eta = 0.03, accuracy = 0.5168
adagrad : m = 100, eta = 0.005, accuracy = 0.5192
adagrad : m = 100, eta = 0.001, accuracy = 0.4992
adagrad : m = 500, eta = 1.5, accuracy = 0.5726
adagrad : m = 500, eta = 0.25, accuracy = 0.5248
adagrad : m = 500, eta = 0.03, accuracy = 0.5142
adagrad : m = 500, eta = 0.005, accuracy = 0.5008
adagrad : m = 500, eta = 0.001, accuracy = 0.5008
adagrad : m = 1000, eta = 1.5, accuracy = 0.7672
adagrad : m = 1000, eta = 0.25, accuracy = 0.4992
adagrad : m = 1000, eta = 0.03, accuracy = 0.4992
adagrad : m = 1000, eta = 0.005, accuracy = 0.4992
adagrad : m = 1000, eta = 0.001, accuracy = 0.4992


In [11]:
perceptron_params = [1.5,0.05,0.25];
winnow_params = [1.1,1.1,1.1];
winnow_margin_params1 = [1.1,1.1,1.1];
winnow_margin_params2 = [0.3,0.01,0.006];
adagrad_params = [1.5,1.5,1.5];

In [12]:
for i in range(3):
    w,theta = perceptron(train_x[i], train_y[i])
    acc = accuracy(w,theta,test_x[i],test_y[i])
    print("Perceptron : m = {}, accuracy = {}".format(m[i], acc))


Perceptron : m = 100, accuracy = 0.9805
Perceptron : m = 500, accuracy = 0.9079
Perceptron : m = 1000, accuracy = 0.7538


In [13]:
for i in range(3):
    w,theta = perceptron_margin(train_x[i], train_y[i], perceptron_params[i])
    acc = accuracy(w,theta,test_x[i],test_y[i])
    print("Perceptron_margin : m = {}, accuracy = {}".format(m[i], acc))



Perceptron_margin : m = 100, accuracy = 0.9805
Perceptron_margin : m = 500, accuracy = 0.8419
Perceptron_margin : m = 1000, accuracy = 0.8264


In [14]:
for i in range(3):
    w,theta = winnow(train_x[i], train_y[i], winnow_params[i])
    acc = accuracy(w,theta,test_x[i],test_y[i])
    print("Winnow : m = {}, accuracy = {}".format(m[i], acc))



Winnow : m = 100, accuracy = 0.948
Winnow : m = 500, accuracy = 0.8775
Winnow : m = 1000, accuracy = 0.7274


In [15]:
for i in range(3):
    w,theta = winnow_margin(train_x[i], train_y[i], winnow_margin_params1[i], winnow_margin_params2[i])
    acc = accuracy(w,theta,test_x[i],test_y[i])
    print("Winnow margin : m = {}, accuracy = {}".format(m[i], acc))



Winnow margin : m = 100, accuracy = 0.9516
Winnow margin : m = 500, accuracy = 0.9084
Winnow margin : m = 1000, accuracy = 0.7146


In [16]:
for i in range(3):
    w,theta = adagrad(train_x[i], train_y[i], adagrad_params[i])
    acc = accuracy(w,theta,test_x[i],test_y[i])
    print("Adagrad : m = {}, accuracy = {}".format(m[i], acc))



Adagrad : m = 100, accuracy = 0.9323
Adagrad : m = 500, accuracy = 0.7716
Adagrad : m = 1000, accuracy = 0.8251
